<font color="#de3023"><h1><b>REMINDER: MAKE A COPY OF THIS NOTEBOOK, DO NOT EDIT</b></h1></font>

# Lab 2: Tensorflow Basics

This tutorial is based on original work from the Tensorflow team at Google. For more information, please visit https://www.tensorflow.org/tutorials/quickstart/beginner

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Load a prebuilt dataset.
1. Build a neural network machine learning model that classifies images.
2. Train this neural network.
3. Evaluate the accuracy of the model.

This tutorial is a [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) notebook. Python programs are run directly in the browser—a great way to learn and use TensorFlow. To follow this tutorial, run the notebook in Google Colab by clicking the button at the top of this page.

1. In Colab, connect to a Python runtime: At the top-right of the menu bar, select *CONNECT*.
2. Run all the notebook code cells: Select *Runtime* > *Run all*.

## Set up TensorFlow

Import TensorFlow into your program to get started:

In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.8.2


## Load a dataset

Don't worry about the details in the cells below: we will get to understanding this later! Just run it for now: the goal is to understand the overall "machine learning framework"

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the sample data from integers to floating-point numbers:

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11501568/11490434 [==============================] - 0s 0us/step


## Build a machine learning model

Build a `tf.keras.Sequential` model by stacking layers.

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example, the model returns a vector of [logits](https://developers.google.com/machine-learning/glossary#logits) or [log-odds](https://developers.google.com/machine-learning/glossary#log-odds) scores, one for each class.

In [ ]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.3825433 ,  0.43683448, -0.9142511 , -0.3739995 ,  0.05886851,
         1.0930117 ,  0.1358074 , -0.5331402 ,  0.58164054,  0.11829735]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to *probabilities* for each class: 

In [ ]:
tf.nn.softmax(predictions).numpy()

array([[0.05679935, 0.12888269, 0.03337534, 0.05728671, 0.08831736,
        0.24840982, 0.09538063, 0.04885848, 0.1489646 , 0.09372505]],
      dtype=float32)

Note: It is possible to bake the `tf.nn.softmax` function into the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation for all models when using a softmax output. 

Define a loss function for training using `losses.SparseCategoricalCrossentropy`, which takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [ ]:
loss_fn(y_train[:1], predictions).numpy()

1.3926754

Before you start training, configure and compile the model using Keras `Model.compile`. Set the [`optimizer`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) class to `adam`, set the `loss` to the `loss_fn` function you defined earlier, and specify a metric to be evaluated for the model by setting the `metrics` parameter to `accuracy`.

In [ ]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## Train and evaluate your model

Use the `Model.fit` method to adjust your model parameters and minimize the loss: 

In [ ]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 17s 8ms/step - loss: 0.2942 - accuracy: 0.9149
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1459 - accuracy: 0.9571
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1070 - accuracy: 0.9674
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0872 - accuracy: 0.9725
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0741 - accuracy: 0.9764


The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" or "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

In [ ]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - loss: 0.0724 - accuracy: 0.9766 - 617ms/epoch - 2ms/step


[0.07239168882369995, 0.9765999913215637]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [ ]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [ ]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[2.7399057e-07, 5.6684732e-09, 8.6002648e-05, 8.5093436e-04,
        2.9084143e-11, 3.0526692e-08, 5.7742347e-13, 9.9905258e-01,
        8.7816642e-07, 9.3102790e-06],
       [2.0245350e-08, 2.3663337e-05, 9.9981397e-01, 1.0233407e-05,
        7.1290435e-15, 2.4797473e-07, 6.5187498e-07, 1.6457458e-13,
        1.5135157e-04, 5.0368515e-13],
       [1.0176991e-06, 9.9871123e-01, 3.9345065e-05, 8.6438868e-06,
        1.9107199e-05, 4.5304787e-06, 2.3921042e-05, 1.0108195e-03,
        1.7855881e-04, 2.8352119e-06],
       [9.9986148e-01, 5.5997101e-10, 5.2587689e-05, 5.6833660e-06,
        1.4511302e-06, 3.0030087e-06, 5.3288575e-05, 7.3009342e-06,
        3.6012554e-07, 1.4846852e-05],
       [2.4630356e-06, 1.0663855e-09, 4.0574841e-06, 4.6370781e-07,
        9.8725116e-01, 1.6383042e-06, 6.8636655e-06, 1.3454043e-04,
        4.7457647e-06, 1.2594026e-02]], dtype=float32)>

## Conclusion

Congratulations! You have trained a machine learning model using a prebuilt dataset using the [Keras](https://www.tensorflow.org/guide/keras/overview) API.

For more examples of using Keras, check out the [tutorials](https://www.tensorflow.org/tutorials/keras/). To learn more about building models with Keras, read the [guides](https://www.tensorflow.org/guide/keras). If you want learn more about loading and preparing data, see the tutorials on [image data loading](https://www.tensorflow.org/tutorials/load_data/images) or [CSV data loading](https://www.tensorflow.org/tutorials/load_data/csv).


# Introduction to Tensors

In [ ]:
import tensorflow as tf
import numpy as np

Tensors are multi-dimensional arrays with a uniform type (called a `dtype`).  You can see all supported `dtypes` at `tf.dtypes.DType`.

If you're familiar with [NumPy](https://numpy.org/devdocs/user/quickstart.html){:.external}, tensors are (kind of) like `np.arrays`.

All tensors are immutable like Python numbers and strings: you can never update the contents of a tensor, only create a new one.


## Basics

First, create some basic tensors.

Here is a "scalar" or "rank-0" tensor . A scalar contains a single value, and no "axes".

In [ ]:
# This will be an int32 tensor by default; see "dtypes" below.
rank_0_tensor = tf.constant(4)
print(rank_0_tensor)

tf.Tensor(4, shape=(), dtype=int32)


A "vector" or "rank-1" tensor is like a list of values. A vector has one axis:

In [ ]:
# Let's make this a float tensor.
rank_1_tensor = tf.constant([2.0, 3.0, 4.0])
print(rank_1_tensor)

tf.Tensor([2. 3. 4.], shape=(3,), dtype=float32)


In [ ]:
#############################################################################
# Task
# • Construct a rank-1 tensor of ones (tf.ones) of length 52.
#############################################################################

# Replace "____" statements with your code
x = tf.ones(52)
print(x.shape)

(52,)


A "matrix" or "rank-2" tensor has two axes:

In [ ]:
# If you want to be specific, you can set the dtype (see below) at creation time
rank_2_tensor = tf.constant([[1, 2],
                             [3, 4],
                             [5, 6]], dtype=tf.float16)
print(rank_2_tensor)

<table>
<tr>
  <th>A scalar, shape: <code>[]</code></th>
  <th>A vector, shape: <code>[3]</code></th>
  <th>A matrix, shape: <code>[3, 2]</code></th>
</tr>
<tr>
  <td>
   <img src="https://github.com/tensorflow/docs/blob/master/site/en/guide/images/tensor/scalar.png?raw=1" alt="A scalar, the number 4" />
  </td>

  <td>
   <img src="https://github.com/tensorflow/docs/blob/master/site/en/guide/images/tensor/vector.png?raw=1" alt="The line with 3 sections, each one containing a number."/>
  </td>
  <td>
   <img src="https://github.com/tensorflow/docs/blob/master/site/en/guide/images/tensor/matrix.png?raw=1" alt="A 3x2 grid, with each cell containing a number.">
  </td>
</tr>
</table>


In [ ]:
#############################################################################
# Task
# • Construct a rank-2 tensor of random (tf.random.uniform) of shape 3 x 9. Cast
#   the x to be of dtype float64
#############################################################################

# Replace "____" statements with your code
x = tf.random.uniform((3, 9), dtype=tf.float64)
print(x.shape)
print(x.dtype)

(3, 9)
<dtype: 'float64'>


Tensors may have more axes; here is a tensor with three axes:

In [ ]:
# There can be an arbitrary number of
# axes (sometimes called "dimensions")
rank_3_tensor = tf.constant([
  [[0, 1, 2, 3, 4],
   [5, 6, 7, 8, 9]],
  [[10, 11, 12, 13, 14],
   [15, 16, 17, 18, 19]],
  [[20, 21, 22, 23, 24],
   [25, 26, 27, 28, 29]],])

print(rank_3_tensor)

There are many ways you might visualize a tensor with more than two axes.

<table>
<tr>
  <th colspan=3>A 3-axis tensor, shape: <code>[3, 2, 5]</code></th>
<tr>
<tr>
  <td>
   <img src="https://github.com/tensorflow/docs/blob/master/site/en/guide/images/tensor/3-axis_numpy.png?raw=1"/>
  </td>
  <td>
   <img src="https://github.com/tensorflow/docs/blob/master/site/en/guide/images/tensor/3-axis_front.png?raw=1"/>
  </td>

  <td>
   <img src="https://github.com/tensorflow/docs/blob/master/site/en/guide/images/tensor/3-axis_block.png?raw=1"/>
  </td>
</tr>

</table>

In [ ]:
#############################################################################
# Task
# • Construct a rank-3 tensor of your choice of shape 3 x 2 x 7. Cast
#   the x to be of dtype float32
#############################################################################

# Replace "____" statements with your code
x = tf.random.uniform((3, 2, 7), dtype=tf.float32)
print(x.shape)
print(x.dtype)

(3, 2, 7)
<dtype: 'float32'>


You can convert a tensor to a NumPy array either using `np.array` or the `tensor.numpy` method:

In [ ]:
np.array(x)

array([[[0.311082  , 0.50559056, 0.47349155, 0.6141002 , 0.564646  ,
         0.77227366, 0.24908972],
        [0.47303426, 0.34400034, 0.6103295 , 0.22859323, 0.0718596 ,
         0.9078343 , 0.9865426 ]],

       [[0.85584104, 0.95000136, 0.97428644, 0.6818764 , 0.11070645,
         0.20882797, 0.0457325 ],
        [0.06053221, 0.23128831, 0.2702732 , 0.42043543, 0.25761425,
         0.6038022 , 0.6193241 ]],

       [[0.46269453, 0.30842924, 0.8958653 , 0.6990013 , 0.76113975,
         0.7322353 , 0.91755545],
        [0.3343941 , 0.79109776, 0.90356636, 0.76219904, 0.63332546,
         0.6940831 , 0.3775773 ]]], dtype=float32)

In [ ]:
#############################################################################
# Task
# • Call the numpy() method on x to convert from a tf.Tensor to a np.Array
#############################################################################

# Replace "____" statements with your code
print(x.numpy())

[[[0.311082   0.50559056 0.47349155 0.6141002  0.564646   0.77227366
   0.24908972]
  [0.47303426 0.34400034 0.6103295  0.22859323 0.0718596  0.9078343
   0.9865426 ]]

 [[0.85584104 0.95000136 0.97428644 0.6818764  0.11070645 0.20882797
   0.0457325 ]
  [0.06053221 0.23128831 0.2702732  0.42043543 0.25761425 0.6038022
   0.6193241 ]]

 [[0.46269453 0.30842924 0.8958653  0.6990013  0.76113975 0.7322353
   0.91755545]
  [0.3343941  0.79109776 0.90356636 0.76219904 0.63332546 0.6940831
   0.3775773 ]]]


Tensors often contain floats and ints, but have many other types, including:

* complex numbers
* strings

The base `tf.Tensor` class requires tensors to be "rectangular"---that is, along each axis, every element is the same size.  However, there are specialized types of tensors that can handle different shapes:

* Ragged tensors (see [RaggedTensor](#ragged_tensors) below)
* Sparse tensors (see [SparseTensor](#sparse_tensors) below)

You can do basic math on tensors, including addition, element-wise multiplication, and matrix multiplication.

In [ ]:
a = tf.constant([[1, 2],
                 [3, 4]])
b = tf.constant([[1, 1],
                 [1, 1]]) # Could have also said `tf.ones([2,2])`

print(tf.add(a, b), "\n")
print(tf.multiply(a, b), "\n")
print(tf.matmul(a, b), "\n")

tf.Tensor(
[[2 3]
 [4 5]], shape=(2, 2), dtype=int32) 

tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32) 

tf.Tensor(
[[3 3]
 [7 7]], shape=(2, 2), dtype=int32) 



In [ ]:
print(a + b, "\n") # element-wise addition
print(a * b, "\n") # element-wise multiplication
print(a @ b, "\n") # matrix multiplication

tf.Tensor(
[[2 3]
 [4 5]], shape=(2, 2), dtype=int32) 

tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32) 

tf.Tensor(
[[3 3]
 [7 7]], shape=(2, 2), dtype=int32) 



In [ ]:
#############################################################################
# Task (numpy -> TF review)
# • Print the shape of the *matrix multiplication* of x and y. What shape do you expect?
# • Print the shape of the *matrix multiplication* of x, y, and z. What shape do you expect?
#############################################################################

# Replace "____" statements with your code
x = tf.convert_to_tensor(np.random.random((12, 19)))
y = tf.convert_to_tensor(np.random.random((19, 21)))
z = tf.convert_to_tensor(np.random.random((21, 23)))

print((x @ y).shape)
print((x @ y @ z).shape)

(12, 21)
(12, 23)


Tensors are used in all kinds of operations (or "Ops").

In [ ]:
c = tf.constant([[4.0, 5.0], [10.0, 1.0]])

# Find the largest value
print(tf.reduce_max(c))
# Find the index of the largest value
print(tf.math.argmax(c))
# Compute the softmax
print(tf.nn.softmax(c))

tf.Tensor(10.0, shape=(), dtype=float32)
tf.Tensor([1 0], shape=(2,), dtype=int64)
tf.Tensor(
[[2.6894143e-01 7.3105860e-01]
 [9.9987662e-01 1.2339458e-04]], shape=(2, 2), dtype=float32)


Note: Typically, anywhere a TensorFlow function expects a `Tensor` as input, the function will also accept anything that can be converted to a `Tensor` using `tf.convert_to_tensor`. See below for an example.

In [ ]:
tf.convert_to_tensor([1,2,3])

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([1, 2, 3], dtype=int32)>

In [ ]:
tf.reduce_max([1,2,3])

<tf.Tensor: shape=(), dtype=int32, numpy=3>

In [ ]:
tf.reduce_max(np.array([1,2,3]))

<tf.Tensor: shape=(), dtype=int64, numpy=3>

In [ ]:
#############################################################################
# Task
# Search google/documentation to find functions for the following
# • Compute the sum of the elements of x
# • Compute the average of the sum of x and y
# • Compute the min of the matrix multiply of x, y, and z
#############################################################################

# Replace "____" statements with your code
x = tf.convert_to_tensor(np.random.random((12, 12)))
y = tf.convert_to_tensor(np.random.random((12, 12)))
z = tf.convert_to_tensor(np.random.random((12, 19)))

print(tf.reduce_sum(x))
print(tf.reduce_mean(x + y))
print(tf.reduce_min(x @ y @ z))

tf.Tensor(73.35398270231467, shape=(), dtype=float64)
tf.Tensor(0.9439676543203634, shape=(), dtype=float64)
tf.Tensor(10.819173118405008, shape=(), dtype=float64)


## About shapes

Tensors have shapes.  Some vocabulary:

* **Shape**: The length (number of elements) of each of the axes of a tensor.
* **Rank**: Number of tensor axes.  A scalar has rank 0, a vector has rank 1, a matrix is rank 2.
* **Axis** or **Dimension**: A particular dimension of a tensor.
* **Size**: The total number of items in the tensor, the product of the shape vector's elements.


Note: Although you may see reference to a "tensor of two dimensions", a rank-2 tensor does not usually describe a 2D space.

Tensors and `tf.TensorShape` objects have convenient properties for accessing these:

In [ ]:
rank_4_tensor = tf.zeros([3, 2, 4, 5])

<table>
<tr>
  <th colspan=2>A rank-4 tensor, shape: <code>[3, 2, 4, 5]</code></th>
</tr>
<tr>
  <td>
<img src="https://github.com/tensorflow/docs/blob/master/site/en/guide/images/tensor/shape.png?raw=1" alt="A tensor shape is like a vector.">
    <td>
<img src="https://github.com/tensorflow/docs/blob/master/site/en/guide/images/tensor/4-axis_block.png?raw=1" alt="A 4-axis tensor">
  </td>
  </tr>
</table>


In [ ]:
print("Type of every element:", rank_4_tensor.dtype)
print("Number of axes:", rank_4_tensor.ndim)
print("Shape of tensor:", rank_4_tensor.shape)
print("Elements along axis 0 of tensor:", rank_4_tensor.shape[0])
print("Elements along the last axis of tensor:", rank_4_tensor.shape[-1])
print("Total number of elements (3*2*4*5): ", tf.size(rank_4_tensor).numpy())

Type of every element: <dtype: 'float32'>
Number of axes: 4
Shape of tensor: (3, 2, 4, 5)
Elements along axis 0 of tensor: 3
Elements along the last axis of tensor: 5
Total number of elements (3*2*4*5):  120


But note that the `Tensor.ndim` and `Tensor.shape` attributes don't return `Tensor` objects. If you need a `Tensor` use the `tf.rank` or `tf.shape` function. This difference is subtle, but it can be important when building graphs (later).

In [ ]:
tf.rank(rank_4_tensor)

<tf.Tensor: shape=(), dtype=int32, numpy=4>

In [ ]:
tf.shape(rank_4_tensor)

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([3, 2, 4, 5], dtype=int32)>

While axes are often referred to by their indices, you should always keep track of the meaning of each. Often axes are ordered from global to local: The batch axis first, followed by spatial dimensions, and features for each location last. This way feature vectors are contiguous regions of memory.

<table>
<tr>
<th>Typical axis order</th>
</tr>
<tr>
    <td>
<img src="https://github.com/tensorflow/docs/blob/master/site/en/guide/images/tensor/shape2.png?raw=1" alt="Keep track of what each axis is. A 4-axis tensor might be: Batch, Width, Height, Features">
  </td>
</tr>
</table>

## Indexing

### Single-axis indexing

TensorFlow follows standard Python indexing rules, similar to [indexing a list or a string in Python](https://docs.python.org/3/tutorial/introduction.html#strings){:.external}, and the basic rules for NumPy indexing.

* indexes start at `0`
* negative indices count backwards from the end
* colons, `:`, are used for slices: `start:stop:step`


In [ ]:
rank_1_tensor = tf.constant([0, 1, 1, 2, 3, 5, 8, 13, 21, 34])
print(rank_1_tensor.numpy())

[ 0  1  1  2  3  5  8 13 21 34]


Indexing with a scalar removes the axis:

In [ ]:
print("First:", rank_1_tensor[0].numpy())
print("Second:", rank_1_tensor[1].numpy())
print("Last:", rank_1_tensor[-1].numpy())

First: 0
Second: 1
Last: 34


Indexing with a `:` slice keeps the axis:

In [ ]:
print("Everything:", rank_1_tensor[:].numpy())
print("Before 4:", rank_1_tensor[:4].numpy())
print("From 4 to the end:", rank_1_tensor[4:].numpy())
print("From 2, before 7:", rank_1_tensor[2:7].numpy())
print("Every other item:", rank_1_tensor[::2].numpy())
print("Reversed:", rank_1_tensor[::-1].numpy())

Everything: [ 0  1  1  2  3  5  8 13 21 34]
Before 4: [0 1 1 2]
From 4 to the end: [ 3  5  8 13 21 34]
From 2, before 7: [1 2 3 5 8]
Every other item: [ 0  1  3  8 21]
Reversed: [34 21 13  8  5  3  2  1  1  0]


In [ ]:
#############################################################################
# Task
# • Print the shape of test (confirm it's what you'd expect from the initialization)
# • Print the entire range with 0 fixed as the 1st index of x. What should its shape be?
# • Print the entire range with 2 fixed as the 3rd index x. What should its shape be?
# • Print the (inclusive) range 1-3, 4-5, 0-1 for the three axes. What should its shape be?
# • Print the "bottom right" element of x.
#############################################################################

# Replace "____" statements with your code
x = tf.convert_to_tensor(np.random.random((12, 19, 3)))

first_slice = x[0, :, :]
print(first_slice.shape)

second_slice = x[:, :, 2]
print(second_slice.shape)

third_slice = x[1:4, 4:6, 0:2]
print(third_slice.shape)

fourth_slice = x[-1, -1, -1]
print(fourth_slice.shape)

(19, 3)
(12, 19)
(3, 2, 2)
()


### Multi-axis indexing

Higher rank tensors are indexed by passing multiple indices.

The exact same rules as in the single-axis case apply to each axis independently.

In [ ]:
rank_2_tensor = tf.convert_to_tensor(np.random.random((12, 3)))
rank_3_tensor = tf.convert_to_tensor(np.random.random((4, 9, 6)))

In [ ]:
print(rank_2_tensor.numpy())

[[0.36124628 0.01783757 0.50199621]
 [0.44142403 0.30519811 0.18009809]
 [0.86875719 0.06310044 0.90469594]
 [0.76525894 0.7967007  0.39074241]
 [0.26388015 0.39740138 0.15184722]
 [0.49802373 0.48115698 0.60253561]
 [0.71694084 0.95357992 0.8184366 ]
 [0.76848335 0.44690603 0.71023938]
 [0.46565172 0.40850682 0.0953134 ]
 [0.75082052 0.14786433 0.94500588]
 [0.97035554 0.19144161 0.22200538]
 [0.35891763 0.61890664 0.436866  ]]


Passing an integer for each index, the result is a scalar.

In [ ]:
# Pull out a single value from a 2-rank tensor
print(rank_2_tensor[1, 1].numpy())

0.3051981060597251


You can index using any combination of integers and slices:

In [ ]:
# Get row and column tensors
print("Second row:", rank_2_tensor[1, :].numpy())
print("Second column:", rank_2_tensor[:, 1].numpy())
print("Last row:", rank_2_tensor[-1, :].numpy())
print("First item in last column:", rank_2_tensor[0, -1].numpy())
print("Skip the first row:")
print(rank_2_tensor[1:, :].numpy(), "\n")

Second row: [0.44142403 0.30519811 0.18009809]
Second column: [0.01783757 0.30519811 0.06310044 0.7967007  0.39740138 0.48115698
 0.95357992 0.44690603 0.40850682 0.14786433 0.19144161 0.61890664]
Last row: [0.35891763 0.61890664 0.436866  ]
First item in last column: 0.5019962113744904
Skip the first row:
[[0.44142403 0.30519811 0.18009809]
 [0.86875719 0.06310044 0.90469594]
 [0.76525894 0.7967007  0.39074241]
 [0.26388015 0.39740138 0.15184722]
 [0.49802373 0.48115698 0.60253561]
 [0.71694084 0.95357992 0.8184366 ]
 [0.76848335 0.44690603 0.71023938]
 [0.46565172 0.40850682 0.0953134 ]
 [0.75082052 0.14786433 0.94500588]
 [0.97035554 0.19144161 0.22200538]
 [0.35891763 0.61890664 0.436866  ]] 



Here is an example with a 3-axis tensor:

In [ ]:
print(rank_3_tensor[:, :, 4])

tf.Tensor(
[[0.49869657 0.15430843 0.32458926 0.77938748 0.22085454 0.22539101
  0.03765203 0.31648853 0.301776  ]
 [0.42880267 0.49762873 0.31585144 0.32804953 0.7004888  0.63329688
  0.93385607 0.71089496 0.67980601]
 [0.07848038 0.86357673 0.39365326 0.40314203 0.73822043 0.29748037
  0.60156247 0.20713563 0.18289452]
 [0.32717457 0.83198961 0.68813097 0.37149738 0.64376816 0.59581327
  0.45466506 0.38308693 0.66528682]], shape=(4, 9), dtype=float64)


<table>
<tr>
<th colspan=2>Selecting the last feature across all locations in each example in the batch </th>
</tr>
<tr>
    <td>
<img src="https://github.com/tensorflow/docs/blob/master/site/en/guide/images/tensor/index1.png?raw=1" alt="A 3x2x5 tensor with all the values at the index-4 of the last axis selected.">
  </td>
      <td>
<img src="https://github.com/tensorflow/docs/blob/master/site/en/guide/images/tensor/index2.png?raw=1" alt="The selected values packed into a 2-axis tensor.">
  </td>
</tr>
</table>

Read the [tensor slicing guide](https://tensorflow.org/guide/tensor_slicing) to learn how you can apply indexing to manipulate individual elements in your tensors.

## Manipulating Shapes

Reshaping a tensor is of great utility. 


In [ ]:
# Shape returns a `TensorShape` object that shows the size along each axis
x = tf.constant([[1], [2], [3]])
print(x.shape)

In [ ]:
# You can convert this object into a Python list, too
print(x.shape.as_list())

You can reshape a tensor into a new shape. The `tf.reshape` operation is fast and cheap as the underlying data does not need to be duplicated.

In [ ]:
# You can reshape a tensor to a new shape.
# Note that you're passing in a list
reshaped = tf.reshape(x, [1, 3])

In [ ]:
print(x.shape)
print(reshaped.shape)

The data maintains its layout in memory and a new tensor is created, with the requested shape, pointing to the same data. TensorFlow uses C-style "row-major" memory ordering, where incrementing the rightmost index corresponds to a single step in memory.

In [ ]:
print(rank_3_tensor)

If you flatten a tensor you can see what order it is laid out in memory.

In [ ]:
# A `-1` passed in the `shape` argument says "Whatever fits".
print(tf.reshape(rank_3_tensor, [-1]))

Typically the only reasonable use of `tf.reshape` is to combine or split adjacent axes (or add/remove `1`s).

For this 3x2x5 tensor, reshaping to (3x2)x5 or 3x(2x5) are both reasonable things to do, as the slices do not mix:

In [ ]:
print(tf.reshape(rank_3_tensor, [3*2, 5]), "\n")
print(tf.reshape(rank_3_tensor, [3, -1]))

<table>
<th colspan=3>
Some good reshapes.
</th>
<tr>
  <td>
<img src="https://github.com/tensorflow/docs/blob/master/site/en/guide/images/tensor/reshape-before.png?raw=1" alt="A 3x2x5 tensor">
  </td>
  <td>
  <img src="https://github.com/tensorflow/docs/blob/master/site/en/guide/images/tensor/reshape-good1.png?raw=1" alt="The same data reshaped to (3x2)x5">
  </td>
  <td>
<img src="https://github.com/tensorflow/docs/blob/master/site/en/guide/images/tensor/reshape-good2.png?raw=1" alt="The same data reshaped to 3x(2x5)">
  </td>
</tr>
</table>


Reshaping will "work" for any new shape with the same total number of elements, but it will not do anything useful if you do not respect the order of the axes.

Swapping axes in `tf.reshape` does not work; you need `tf.transpose` for that. 


In [ ]:
# Bad examples: don't do this

# You can't reorder axes with reshape.
print(tf.reshape(rank_3_tensor, [2, 3, 5]), "\n") 

# This is a mess
print(tf.reshape(rank_3_tensor, [5, 6]), "\n")

# This doesn't work at all
try:
  tf.reshape(rank_3_tensor, [7, -1])
except Exception as e:
  print(f"{type(e).__name__}: {e}")

<table>
<th colspan=3>
Some bad reshapes.
</th>
<tr>
  <td>
<img src="https://github.com/tensorflow/docs/blob/master/site/en/guide/images/tensor/reshape-bad.png?raw=1" alt="You can't reorder axes, use tf.transpose for that">
  </td>
  <td>
<img src="https://github.com/tensorflow/docs/blob/master/site/en/guide/images/tensor/reshape-bad4.png?raw=1" alt="Anything that mixes the slices of data together is probably wrong.">
  </td>
  <td>
<img src="https://github.com/tensorflow/docs/blob/master/site/en/guide/images/tensor/reshape-bad2.png?raw=1" alt="The new shape must fit exactly.">
  </td>
</tr>
</table>

Note: `tf.transpose` and `tf.reshape` seem quite similar, and may coincide with what you want to do, but a `.reshape` effectively flattens the array (from the inner most to the outer most) and then groups values according to the new shape. This may reshuffle the values in a way you don't want, but `.transpose` effectively just changes the order in which you iterate through the values:

![](https://lihan.me/assets/images/numpy-arrays.png)

`.transpose` requires that you specify *which* dimension goes where. This should be more clear after a couple example exercises:

In [ ]:
N, T, D = 4, 60, 3
cube = tf.Variable(tf.zeros((N, T, D)))

The interface is:
```
tf.transpose(
    a, perm=[x,y,z]
)
```
Where `x,y,z` are the *new* dimension order you want. So, if you specify `perm=[0,2,1]`, this will *switch* the 1st and 2nd dimensions. That means, the output will have dimension `[N, D, T]`. Let's try out some examples:

In [ ]:
#############################################################################
# Task
# • Using transpose, create "rotated_cube" to be D x N x T
#############################################################################

# Replace "____" statements with your code
rotated_cube = tf.transpose(cube, perm=[2,0,1])

print(rotated_cube.shape)

(3, 4, 60)


In [ ]:
#############################################################################
# Task
# • Using transpose, create "rotated_cube" to be T x N x D
#############################################################################

# Replace "____" statements with your code
rotated_cube = tf.transpose(cube, perm=[1,0,2])

print(rotated_cube.shape)

(60, 4, 3)


# Introduction to Variables

A TensorFlow **variable** is the recommended way to represent shared, persistent state your program manipulates. This guide covers how to create, update, and manage instances of `tf.Variable` in TensorFlow.

Variables are created and tracked via the `tf.Variable` class. A `tf.Variable` represents a tensor whose value can be changed by running ops on it.  Specific ops allow you to read and modify the values of this tensor. Higher level libraries like `tf.keras` use `tf.Variable` to store model parameters. 

## Setup

This notebook discusses variable placement.  If you want to see on what device your variables are placed, uncomment this line.

In [ ]:
import tensorflow as tf

## Create a variable

To create a variable, provide an initial value.  The `tf.Variable` will have the same `dtype` as the initialization value.

In [ ]:
my_tensor = tf.constant([[1.0, 2.0], [3.0, 4.0]])
my_variable = tf.Variable(my_tensor)

# Variables can be all kinds of types, just like tensors
bool_variable = tf.Variable([False, False, False, True])
complex_variable = tf.Variable([5 + 4j, 6 + 1j])

A variable looks and acts like a tensor, and, in fact, is a data structure backed by a `tf.Tensor`.  Like tensors, they have a `dtype` and a shape, and can be exported to NumPy.

In [ ]:
print("Shape: ", my_variable.shape)
print("DType: ", my_variable.dtype)
print("As NumPy: ", my_variable.numpy())

Shape:  (2, 2)
DType:  <dtype: 'float32'>
As NumPy:  [[1. 2.]
 [3. 4.]]


Most tensor operations work on variables as expected, although variables cannot be reshaped.

In [ ]:
print("A variable:", my_variable)
print("\nViewed as a tensor:", tf.convert_to_tensor(my_variable))
print("\nIndex of highest value:", tf.math.argmax(my_variable))

# This creates a new tensor; it does not reshape the variable.
print("\nCopying and reshaping: ", tf.reshape(my_variable, [1,4]))

A variable: <tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[1., 2.],
       [3., 4.]], dtype=float32)>

Viewed as a tensor: tf.Tensor(
[[1. 2.]
 [3. 4.]], shape=(2, 2), dtype=float32)

Index of highest value: tf.Tensor([1 1], shape=(2,), dtype=int64)

Copying and reshaping:  tf.Tensor([[1. 2. 3. 4.]], shape=(1, 4), dtype=float32)


As noted above, variables are backed by tensors. You can reassign the tensor using `tf.Variable.assign`.  Calling `assign` does not (usually) allocate a new tensor; instead, the existing tensor's memory is reused.

In [ ]:
a = tf.Variable([2.0, 3.0])
# This will keep the same dtype, float32
a.assign([1, 2]) 
# Not allowed as it resizes the variable: 
try:
  a.assign([1.0, 2.0, 3.0])
except Exception as e:
  print(f"{type(e).__name__}: {e}")

ValueError: Cannot assign value to variable ' Variable:0': Shape mismatch.The variable shape (2,), and the assigned value shape (3,) are incompatible.


In [ ]:
#############################################################################
# Task
# • Try assigning position 1 of a to 6.0 using indexing. What happens?
#############################################################################

# Replace "____" statements with your code
a[1] = 6.0

print(a)

TypeError: ignored

If you use a variable like a tensor in operations, you will usually operate on the backing tensor.  

Creating new variables from existing variables duplicates the backing tensors. Two variables will not share the same memory.

In [ ]:
a = tf.Variable([2.0, 3.0])
# Create b based on the value of a
b = tf.Variable(a)
a.assign([5, 6])

# a and b are different
print(a.numpy())
print(b.numpy())

# There are other versions of assign
print(a.assign_add([2,3]).numpy())  # [7. 9.]
print(a.assign_sub([7,9]).numpy())  # [0. 0.]

In [ ]:
#############################################################################
# Task
# • Construct a variable with value [1,2,3]
# • Construct a variable of size 2 x 3 initialized randomly (either use the
#   tf.random.uniform function or the equivalent from numpy)
#############################################################################

# Replace "____" statements with your code
x = tf.Variable([1,2,3])
print(x)

y = tf.Variable(tf.random.uniform((2, 3)))
print(y)

<tf.Variable 'Variable:0' shape=(3,) dtype=int32, numpy=array([1, 2, 3], dtype=int32)>
<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[0.24620402, 0.8758352 , 0.3004129 ],
       [0.11651826, 0.05574834, 0.42816496]], dtype=float32)>


## Lifecycles, naming, and watching

In Python-based TensorFlow, `tf.Variable` instance have the same lifecycle as other Python objects. When there are no references to a variable it is automatically deallocated.

Variables can also be named which can help you track and debug them.  You can give two variables the same name.

In [ ]:
# Create a and b; they will have the same name but will be backed by
# different tensors.
a = tf.Variable(my_tensor, name="Mark")
# A new variable with the same name, but different value
# Note that the scalar add is broadcast
b = tf.Variable(my_tensor + 1, name="Mark")

# These are elementwise-unequal, despite having the same name
print(a == b)

tf.Tensor(
[[False False]
 [False False]], shape=(2, 2), dtype=bool)


Variable names are preserved when saving and loading models. By default, variables in models will acquire unique variable names automatically, so you don't need to assign them yourself unless you want to.

Although variables are important for differentiation, some variables will not need to be differentiated.  You can turn off gradients for a variable by setting `trainable` to false at creation. An example of a variable that would not need gradients is a training step counter.

In [ ]:
step_counter = tf.Variable(1, trainable=False)

# The Sequential model

## Setup

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## When to use a Sequential model

A `Sequential` model is appropriate for **a plain stack of layers**
where each layer has **exactly one input tensor and one output tensor**.

Schematically, the following `Sequential` model:

In [ ]:
# Define Sequential model with 3 layers
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu", name="layer1"),
        layers.Dense(3, activation="relu", name="layer2"),
        layers.Dense(4, name="layer3"),
    ]
)
# Call model on a test input
x = tf.ones((3, 3))
y = model(x)

is equivalent to this function:

In [ ]:
# Create 3 layers
layer1 = layers.Dense(2, activation="relu", name="layer1")
layer2 = layers.Dense(3, activation="relu", name="layer2")
layer3 = layers.Dense(4, name="layer3")

# Call layers on a test input
x = tf.ones((3, 3))
y = layer3(layer2(layer1(x)))

A Sequential model is **not appropriate** when:

- Your model has multiple inputs or multiple outputs
- Any of your layers has multiple inputs or multiple outputs
- You need to do layer sharing
- You want non-linear topology (e.g. a residual connection, a multi-branch
model)

## Creating a Sequential model

You can create a Sequential model by passing a list of layers to the Sequential
constructor:

In [ ]:
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu"),
        layers.Dense(3, activation="relu"),
        layers.Dense(4),
    ]
)

Its layers are accessible via the `layers` attribute:

In [ ]:
model.layers

In [ ]:
#############################################################################
# Task
# • Construct a Sequential model with 2 Dense layers of sizes 3 and 7
#############################################################################

# Replace "____" statements with your code
model = keras.Sequential(
    [
        layers.Dense(3, activation="relu"),
        layers.Dense(7),
    ]
)

You can also create a Sequential model incrementally via the `add()` method:

In [ ]:
model = keras.Sequential()
model.add(layers.Dense(2, activation="relu"))
model.add(layers.Dense(3, activation="relu"))
model.add(layers.Dense(4))

In [ ]:
#############################################################################
# Task
# • Construct a Sequential model with 2 Dense layers of sizes 3 and 7 using add
#############################################################################

# Replace "____" statements with your code
model = keras.Sequential()
model.add(layers.Dense(3, activation="relu"))
model.add(layers.Dense(7))

Note that there's also a corresponding `pop()` method to remove layers:
a Sequential model behaves very much like a list of layers.

In [ ]:
model.pop()
print(len(model.layers))  # 2

1


Also note that the Sequential constructor accepts a `name` argument, just like
any layer or model in Keras. This is useful to annotate TensorBoard graphs
with semantically meaningful names.

In [ ]:
model = keras.Sequential(name="my_sequential")
model.add(layers.Dense(2, activation="relu", name="layer1"))
model.add(layers.Dense(3, activation="relu", name="layer2"))
model.add(layers.Dense(4, name="layer3"))

In [ ]:
#############################################################################
# Task
# • Use a for loop to construct a model with layers of the following sizes
#   specified by layer_sizes
# • Look at the properties of the model using model.summary(). How many parameters
#   are there in this model? 
#############################################################################

# Replace "____" statements with your code
layer_sizes = [1, 2, 3, 4, 5, 4, 3, 2, 1]

model = keras.Sequential()

for i, layer_size in enumerate(layer_sizes):
  if i == len(layer_sizes) - 1:
    model.add(layers.Dense(layer_sizes[i]))
  else:
    model.add(layers.Dense(layer_sizes[i], activation="relu"))

model.build(input_shape=(4,5))
model.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_103 (Dense)           (4, 1)                    6         
                                                                 
 dense_104 (Dense)           (4, 2)                    4         
                                                                 
 dense_105 (Dense)           (4, 3)                    9         
                                                                 
 dense_106 (Dense)           (4, 4)                    16        
                                                                 
 dense_107 (Dense)           (4, 5)                    25        
                                                                 
 dense_108 (Dense)           (4, 4)                    24        
                                                                 
 dense_109 (Dense)           (4, 3)                  

## Specifying the input shape in advance

Generally, all layers in Keras need to know the shape of their inputs
in order to be able to create their weights. So when you create a layer like
this, initially, it has no weights:

In [ ]:
layer = layers.Dense(3)
layer.weights  # Empty

[]

It creates its weights the first time it is called on an input, since the shape
of the weights depends on the shape of the inputs:

In [ ]:
# Call layer on a test input
x = tf.ones((1, 4))
y = layer(x)
layer.weights  # Now it has weights, of shape (4, 3) and (3,)

[<tf.Variable 'dense_57/kernel:0' shape=(4, 3) dtype=float32, numpy=
 array([[-0.7688719 ,  0.58455515, -0.22991276],
        [ 0.7622249 ,  0.44078648, -0.23729473],
        [-0.17711347, -0.7536031 , -0.26955682],
        [-0.65907276,  0.4294542 , -0.16519457]], dtype=float32)>,
 <tf.Variable 'dense_57/bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]

Naturally, this also applies to Sequential models. When you instantiate a
Sequential model without an input shape, it isn't "built": it has no weights
(and calling
`model.weights` results in an error stating just this). The weights are created
when the model first sees some input data:

In [ ]:
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu"),
        layers.Dense(3, activation="relu"),
        layers.Dense(4),
    ]
)  # No weights at this stage!

# At this point, you can't do this:
# model.weights

# You also can't do this:
# model.summary()

# Call the model on a test input
x = tf.ones((1, 4))
y = model(x)
print("Number of weights after calling the model:", len(model.weights))  # 6

Number of weights after calling the model: 6


Once a model is "built", you can call its `summary()` method to display its
contents:

In [ ]:
model.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_112 (Dense)           (1, 2)                    10        
                                                                 
 dense_113 (Dense)           (1, 3)                    9         
                                                                 
 dense_114 (Dense)           (1, 4)                    16        
                                                                 
Total params: 35
Trainable params: 35
Non-trainable params: 0
_________________________________________________________________


However, it can be very useful when building a Sequential model incrementally
to be able to display the summary of the model so far, including the current
output shape. In this case, you should start your model by passing an `Input`
object to your model, so that it knows its input shape from the start:

In [ ]:
model = keras.Sequential()
model.add(keras.Input(shape=(4,)))
model.add(layers.Dense(2, activation="relu"))

model.summary()

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_115 (Dense)           (None, 2)                 10        
                                                                 
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


Note that the `Input` object is not displayed as part of `model.layers`, since
it isn't a layer:

In [ ]:
model.layers

A simple alternative is to just pass an `input_shape` argument to your first
layer:

In [ ]:
model = keras.Sequential()
model.add(layers.Dense(2, activation="relu", input_shape=(4,)))

model.summary()

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_116 (Dense)           (None, 2)                 10        
                                                                 
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


Models built with a predefined input shape like this always have weights (even
before seeing any data) and always have a defined output shape.

In general, it's a recommended best practice to always specify the input shape
of a Sequential model in advance if you know what it is.

In [ ]:
#############################################################################
# Task
# • Print the summary iteratively as you add layers as above (using the input_shape given)
#############################################################################

# Replace "____" statements with your code
layer_sizes = [1, 2, 3, 4, 5, 4, 3, 2, 1]

model = keras.Sequential()

for i, layer_size in enumerate(layer_sizes):
  if i == len(layer_sizes) - 1:
    model.add(layers.Dense(layer_sizes[i]))
  elif i == 0:
    model.add(layers.Dense(layer_sizes[i], activation="relu", input_shape=(4,5)))
  else:
    model.add(layers.Dense(layer_sizes[i], activation="relu"))
  model.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_117 (Dense)           (None, 4, 1)              6         
                                                                 
Total params: 6
Trainable params: 6
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_117 (Dense)           (None, 4, 1)              6         
                                                                 
 dense_118 (Dense)           (None, 4, 2)              4         
                                                                 
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_21"
_________________

## A common debugging workflow: `add()` + `summary()`

When building a new Sequential architecture, it's useful to incrementally stack
layers with `add()` and frequently print model summaries. For instance, this
enables you to monitor how a stack of `Conv2D` and `MaxPooling2D` layers is
downsampling image feature maps:

In [ ]:
model = keras.Sequential()
model.add(keras.Input(shape=(250, 250, 3)))  # 250x250 RGB images
model.add(layers.Conv2D(32, 5, strides=2, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(3))

# Can you guess what the current output shape is at this point? Probably not.
# Let's just print it:
model.summary()

# The answer was: (40, 40, 32), so we can keep downsampling...

model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(3))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(2))

# And now?
model.summary()

# Now that we have 4x4 feature maps, time to apply global max pooling.
model.add(layers.GlobalMaxPooling2D())

# Finally, we add a classification layer.
model.add(layers.Dense(10))

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 123, 123, 32)      2432      
                                                                 
 conv2d_1 (Conv2D)           (None, 121, 121, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 40, 40, 32)       0         
 )                                                               
                                                                 
Total params: 11,680
Trainable params: 11,680
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 123, 123, 32)      2432      
                          

# Training Revisited
Let's now revisit the training from the beginning of class. Training involves three things:
- Compiling the model with a loss function
- Training on some `x_train`, `y_train`
- Testing on some `x_test`, `y_test`
Let's walk through an example of doing this

In [ ]:
#############################################################################
# Task
# • Construct a model with 3 Dense layers of the following sizes: 3, 8, 1
#############################################################################

# Replace "____" statements with your code
model = keras.Sequential(
    [
        layers.Dense(3, activation="relu"),
        layers.Dense(8, activation="relu"),
        layers.Dense(1),
    ]
)

Two common loss functions are "mse" (Mean Squared Error) for regression tasks and "sparse_categorical_crossentropy" for classification tasks. There are others, but these are the two primary ones we'll see in this course. Let's try it with "mse"

In [ ]:
#############################################################################
# Task
# • Compile the model using Adam as the optimizer and the mse loss
#############################################################################

# Replace "____" statements with your code
model.compile(optimizer='adam', loss="mse")

In [ ]:
#############################################################################
# Task
# • Train your model on train_x, train_y
#############################################################################

x = np.random.random((100, 5))
y = np.random.random(100)

train_x, test_x = x[:80, :], x[80:, :]
train_y, test_y = y[:80], y[80:]

# Replace "____" statements with your code
model.fit(train_x, train_y)

3/3 [==============================] - 1s 9ms/step - loss: 0.3339


In [ ]:
#############################################################################
# Task
# • Evaluate your model on test_x, test_y
#############################################################################
model.evaluate(test_x, test_y)

1/1 [==============================] - 0s 245ms/step - loss: 0.2879


0.28792625665664673